In [1]:
import tensorflow as tf
import keras 
from keras import backend as K
import numpy as np

from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.applications.inception_v3 import preprocess_input
from keras.utils import to_categorical

from skimage import color, filters, exposure, morphology

from sklearn.metrics import confusion_matrix

import os
import glob

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
train_path = '../data/MESSIDOR_images/filtered/train'
val_path = '../data/MESSIDOR_images/filtered/val'
test_path = '../data/MESSIDOR_images/filtered/test'

In [21]:
train_batches = ImageDataGenerator(rescale=1./255)\
    .flow_from_directory(directory=train_path, target_size=(224, 224), color_mode='rgb', classes=['NoDR','DR'],
                         batch_size=len(glob.glob(train_path+'/*/*')), seed=9)

test_batches = ImageDataGenerator(rescale=1./255)\
    .flow_from_directory(directory=test_path, target_size=(224, 224), color_mode='rgb', classes=['NoDR','DR'],
                         batch_size=len(glob.glob(test_path+'/*/*')), seed=9)

val_batches = ImageDataGenerator(rescale=1./255)\
    .flow_from_directory(directory=val_path, target_size=(224, 224), color_mode='rgb', classes=['NoDR','DR'],
                         batch_size=len(glob.glob(val_path+'/*/*')), seed=9)

Found 829 images belonging to 2 classes.
Found 106 images belonging to 2 classes.
Found 103 images belonging to 2 classes.


In [22]:
train_image, train_label = next(train_batches)
test_image, test_label = next(test_batches)
val_image, val_label = next(val_batches)

In [3]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=(1/255),
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=(1/255),
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=(1/255),
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

In [27]:
flipped_test = []
for image in test_image:
    im = train_datagen.apply_transform(image, flip_horizontal)
    flipped_test.append(im)

NameError: name 'flip_horizontal' is not defined

In [4]:
train_generator = train_datagen.flow_from_directory(train_path,
                                                    target_size=(224,224),
                                                    save_to_dir='../data/MESSIDOR_images/filtered/resample_train')

Found 829 images belonging to 2 classes.


In [6]:
i=0
for batch in train_generator:
    i+=1
    if (i>10):
        break

In [17]:
sample_length = len(glob.glob('../data/MESSIDOR_images/filtered/resample_train/*'))

In [18]:
resample_labels = []
i=0
for label in next(train_generator)[1]:
    resample_labels.append(label)
    i+=1
    if (i>sample_length):
        break

In [20]:
train_generator

In [19]:
len(resample_labels)

32

In [7]:
image, value = next(train_generator)

In [11]:
next(train_generator)[1]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [6]:
train_batches = ImageDataGenerator(rescale=1./255)\
    .flow_from_directory(directory=train_path, target_size=(224, 224), color_mode='rgb', classes=['NoDR','DR'],
                         batch_size=len(glob.glob(train_path+'/*/*')), seed=9)

test_batches = ImageDataGenerator(rescale=1./255)\
    .flow_from_directory(directory=test_path, target_size=(224, 224), color_mode='rgb', classes=['NoDR','DR'],
                         batch_size=len(glob.glob(test_path+'/*/*')), seed=9)

val_batches = ImageDataGenerator(rescale=1./255)\
    .flow_from_directory(directory=val_path, target_size=224, 224), color_mode='rgb', classes=['NoDR','DR'],
                         batch_size=len(glob.glob(val_path+'/*/*')), seed=9)

Found 959 images belonging to 2 classes.
Found 122 images belonging to 2 classes.
Found 119 images belonging to 2 classes.


In [7]:
train_image, train_label = next(train_batches)
test_image, test_label = next(test_batches)
val_image, val_label = next(val_batches)

In [ ]:
def flip_image(image_array):
    img = [np.fliplr(image) for image in image_array]
    return img

In [ ]:
def default_function(image_array):
    img = image_array
    return img

In [ ]:
def flip_images(name_series,output_folder,image_series, function=default_function, append_name=''): 
    """Saves image by diagnosis and applies a function and appends a name if stated"""
    input_name=name_series.values
    output_folder=output_folder
    
    indexer = 0
    
    for imagePath in input_name:
        
        input_array = image_series[indexer]
        
        filtered_image = function(input_array)
        
        img = Image.fromarray(filtered_image)
        
        fullOutPath = os.path.join(output_folder,append_name+imagePath) 
        
        img.save(fullOutPath)
        
        indexer += 1 

In [ ]:
train_gen = train_datagen.flow_from_directory\
    (directory=train_path,
     target_size=(224,224),
     save_to_dir='../data/MESSIDOR_images/filtered/train_resampled')

In [ ]:
val_datagen.flow_from_directory\
    (directory=val_path,
     target_size=(224,224),
     color_mode='rgb',
     classes=['NoDR','DR'],
     save_to_dir='../data/MESSIDOR_images/filtered/val_resampled',
     seed=9)

In [ ]:
for e in range(epochs):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in train_datagen.flow_from_directory\
    (train_path, batch_size=32, target_size=(224, 224)):
        cnn.fit(x_batch, y_batch)
        batches += 1
        if batches >= len(glob.glob(train_path+'/*/*'))*5:
            break

In [ ]:
model.fit(datagen.flow(x_train, y_train, batch_size=32),
          steps_per_epoch=len(x_train) / 32, epochs=epochs)

In [ ]:
i = 0
train_generator = train_datagen.flow_from_directory\
    (directory=train_path,
     target_size=(224,224,3),
     color_mode='rgb',
     classes=['NoDR','DR'],
     save_to_dir='../data/MESSIDOR_images/filtered/train_resampled',
     batch_size=len(glob.glob(train_path+'/*/*')),
     seed=9)

In [ ]:
test_generator = test_datagen.flow_from_directory\
    (directory=test_path,
     target_size=(224,224,3),
     color_mode='rgb',
     classes=['NoDR','DR'],
     save_to_dir='../data/MESSIDOR_images/filtered/test_resampled',
     batch_size=len(glob.glob(train_path+'/*/*')),
     seed=9)

In [ ]:
val_datagen.flow_from_directory\
    (directory=val_path,
     target_size=(224,224),
     color_mode='rgb',
     classes=['NoDR','DR'],
     save_to_dir='../data/MESSIDOR_images/filtered/val_resampled',
     seed=9)

In [ ]:
train_datagen.flow_from_directory\
    (directory=train_path,
     target_size=(224,224),
     save_to_dir='../data/MESSIDOR_images/filtered/train_resampled')

In [ ]:
horizontally_flipped = np.fliplr(rocket_image)
show_image(horizontally_flipped, 'horizontally flipped image')